# Overview
     0) Download Genome Fasta and Annotation GTF
     1) Clean Fasta with Fastp
     2) STAR Generate Genome
     3) STAR Align
     4) Cufflinks

# Download Reference Genome and Annotations
https://uswest.ensembl.org/Taeniopygia_guttata/Info/Index

In [ ]:
mkdir /bigstore/binfo/zebra_finch/
mkdir /bigstore/binfo/zebra_finch/GenomeFasta/
cd /bigstore/binfo/zebra_finch/GenomeFasta/
rsync -av rsync://ftp.ensembl.org/pub/release-96/fasta/taeniopygia_guttata/dna/

In [ ]:
cd /bigstore/binfo/zebra_finch/
rsync -av rsync://ftp.ensembl.org/pub/release-96/gtf/taeniopygia_guttata/

In [ ]:
!for f in /bigstore/binfo/zebra_finch/GenomeFasta/Taeniopygia_guttata.taeGut3.2.4.dna.chromosome* ; do gunzip $f ; done

In [5]:
!cat /bigstore/binfo/zebra_finch/GenomeFasta/Taeniopygia_guttata.taeGut3.2.4.dna.chromosome* > '/bigstore/binfo/zebra_finch/GenomeFasta/Taeniopygia_guttata.taeGut3.2.4.dna.primary_assembly.fa'

# STAR Generate Genome
    https://github.com/alexdobin/STAR
    conda install -c bioconda star

In [ ]:
# STAR Generate Genome
STAR
--runThreadN 32
--runMode genomeGenerate
--genomeDir /bigstore/binfo/zebra_finch/STARgenomeDir
--genomeFastaFiles /bigstore/binfo/zebra_finch/Taeniopygia_guttata.taeGut3.2.4.dna.primary_assembly.fa
--sjdbGTFfile /bigstore/binfo/zebra_finch/Taeniopygia_guttata.taeGut3.2.4.96.gtf

In [6]:
!STAR --runThreadN 32 --runMode genomeGenerate --genomeDir /bigstore/binfo/zebra_finch/STARgenomeDir --genomeFastaFiles /bigstore/binfo/zebra_finch/Taeniopygia_guttata.taeGut3.2.4.dna.primary_assembly.fa --sjdbGTFfile /bigstore/binfo/zebra_finch/Taeniopygia_guttata.taeGut3.2.4.96.gtf

Apr 28 14:38:15 ..... started STAR run
Apr 28 14:38:15 ... starting to generate Genome files
Apr 28 14:38:50 ... starting to sort Suffix Array. This may take a long time...
Apr 28 14:39:01 ... sorting Suffix Array chunks and saving them to disk...
Apr 28 14:43:51 ... loading chunks from disk, packing SA...
Apr 28 14:44:32 ... finished generating suffix array
Apr 28 14:44:32 ... generating Suffix Array index
Apr 28 14:47:50 ... completed Suffix Array index
Apr 28 14:47:50 ..... processing annotations GTF
Apr 28 14:47:54 ..... inserting junctions into the genome indices
Apr 28 14:49:45 ... writing Genome to disk ...
Apr 28 14:49:48 ... writing Suffix Array to disk ...
Apr 28 14:50:04 ... writing SAindex to disk
Apr 28 14:50:09 ..... finished successfully


# Retrieved Files from Caitlin


In [ ]:
!mkdir /bigstore/binfo/zebra_finch/RegionXExpression/
!mkdir /bigstore/binfo/zebra_finch/RegionXExpression/Raw/

In [7]:
!for f in /bigstore/binfo/zebra_finch/RegionXExpression/Raw/*.fastq.gz ; do gunzip $f ; done

# Clean Fasta Files: Fastp
    https://github.com/OpenGene/fastp
    conda install -c bioconda fastp

In [ ]:
!mkdir /bigstore/binfo/zebra_finch/RegionXExpression/Fastp/
!for f in /bigstore/binfo/zebra_finch/RegionXExpression/Raw/*.fastq ; do fastp -i $f -o /bigstore/binfo/zebra_finch/RegionXExpression/Fastp/$f ; done

In [10]:
pth = '/bigstore/binfo/zebra_finch/RegionXExpression/Fastp/'
Blue623x_S6_L001_R2_001 = open(os.path.join(pth,'Blue623X_S6_L001_R1_001.fastq'))

In [33]:
Blue623x_S6_L001_R2_001.

KeyboardInterrupt: 

# STAR Align

In [7]:
# Generate samples list for later use
pth = '/bigstore/binfo/zebra_finch/RegionXExpression/Raw'
import os
import pandas as pd
files = []
for f in os.listdir(pth):
    if '.fastq' in f:
        fn = f.split('_R')[0]
        if not fn in files:
            files.append(fn)
            print(fn)
files = pd.Series(data=files)
files.to_csv('/bigstore/binfo/zebra_finch/Samples.csv',index='False')

Blue623X_S6_L002
Purple418X_S42_L004
Purple418X_S42_L003
LtPink78X_S40_L004
Brown443X_S79_L008
Blue623X_S6_L001
LtPink78X_S40_L003
LtPink82X_S67_L007
Purple405X_S68_L008
LtPink82X_S67_L008
Purple405X_S68_L007
Brown443X_S79_L007


In [ ]:
# STAR Align
cat /bigstore/binfo/zebra_finch/RegionXExpression/Samples.csv | while read line
do
STAR
--runThreadN 32
--runMode alignReads
--genomeDir /bigstore/binfo/zebra_finch/STARgenomeDir
--readFilesIn /bigstore/binfo/zebra_finch/RegionXExpression/Fastp/"$line"_R1_001.fastq.gz /bigstore/binfo/zebra_finch/RegionXExpression/Fastp/"$line"_R2_001.fastq.gz
--outFileNamePrefix /bigstore/binfo/zebra_finch/RegionXExpression/Aligned/$line
--readFilesCommand gunzip -c
--outSAMtype BAM SortedByCoordinate
--quantMode TranscriptomeSAM GeneCounts
--genomeLoad LoadAndKeep
done

In [ ]:
!mkdir /bigstore/binfo/zebra_finch/RegionXExpression/Aligned

In [8]:
!cat /bigstore/binfo/zebra_finch/RegionXExpression/Samples.csv | while read line
do
STAR --runThreadN 32 --runMode alignReads --genomeDir /bigstore/binfo/zebra_finch/STARgenomeDir/ --readFilesIn /bigstore/binfo/zebra_finch/RegionXExpression/Fastp/"$line"_R1_001.fastq /bigstore/binfo/zebra_finch/RegionXExpression/Fastp/"$line"_R2_001.fastq --outFileNamePrefix /bigstore/binfo/zebra_finch/RegionXExpression/Aligned/$line --outSAMtype BAM SortedByCoordinate --quantMode TranscriptomeSAM GeneCounts
done

SyntaxError: invalid syntax (<ipython-input-8-9e83f7606eff>, line 3)

In [37]:
!STAR --runThreadN 20 --genomeDir /bigstore/binfo/zebra_finch/STARgenomeDir/ --readFilesIn /bigstore/binfo/zebra_finch/RegionXExpression/Fastp/Blue623X_S6_L001_R1_001.fastq /bigstore/binfo/zebra_finch/RegionXExpression/Fastp/Blue623X_S6_L001_R2_001.fastq --outFileNamePrefix /bigstore/binfo/zebra_finch/RegionXExpression/Aligned/Blue623X_S6_L001 --outSAMtype BAM SortedByCoordinate --quantMode TranscriptomeSAM GeneCounts

Apr 28 15:30:39 ..... started STAR run
Apr 28 15:30:39 ..... loading genome
Apr 28 15:30:50 ..... started mapping

EXITING because of FATAL ERROR in reads input: short read sequence line: 0
Read Name=@SN860:669:C8F8HACXX:1:2316:19178:67540
Read Sequence====
DEF_readNameLengthMax=50000
DEF_readSeqLengthMax=650

Apr 28 15:31:18 ...... FATAL ERROR, exiting


In [39]:
for file in os.listdir(pth)
print()

['Blue623X_S6_L001_R1_001.fastq',
 'Blue623X_S6_L001_R2_001.fastq',
 'Blue623X_S6_L002_R1_001.fastq',
 'Blue623X_S6_L002_R2_001.fastq',
 'Brown443X_S79_L007_R1_001.fastq',
 'Brown443X_S79_L007_R2_001.fastq',
 'Brown443X_S79_L008_R1_001.fastq',
 'Brown443X_S79_L008_R2_001.fastq',
 'LtPink78X_S40_L003_R1_001.fastq',
 'LtPink78X_S40_L003_R2_001.fastq',
 'LtPink78X_S40_L004_R1_001.fastq',
 'LtPink78X_S40_L004_R2_001.fastq',
 'LtPink82X_S67_L007_R1_001.fastq',
 'LtPink82X_S67_L007_R2_001.fastq',
 'LtPink82X_S67_L008_R1_001.fastq',
 'LtPink82X_S67_L008_R2_001.fastq',
 'Purple405X_S68_L007_R1_001.fastq',
 'Purple405X_S68_L007_R2_001.fastq',
 'Purple405X_S68_L008_R1_001.fastq',
 'Purple405X_S68_L008_R2_001.fastq',
 'Purple418X_S42_L003_R1_001.fastq',
 'Purple418X_S42_L003_R2_001.fastq',
 'Purple418X_S42_L004_R1_001.fastq',
 'Purple418X_S42_L004_R2_001.fastq']

In [41]:
!cd /bigstore/binfo/zebra_finch/RegionXExpression/Fastp/
!for f in /*.fastq ; do STAR --runThreadN 20 --genomeDir /bigstore/binfo/zebra_finch/STARgenomeDir/ --readFilesIn $f --outFileNamePrefix /bigstore/binfo/zebra_finch/RegionXExpression/Aligned/$f --outSAMtype BAM SortedByCoordinate --quantMode TranscriptomeSAM GeneCounts ; done


EXITING because of fatal input ERROR: could not open readFilesIn=fastp.html

Apr 28 16:24:08 ...... FATAL ERROR, exiting


In [75]:
! cd
!for a in /bigstore/binfo/zebra_finch/RegionXExpression/Fastp/* ; do echo "$(cut -d'_' -f2 <<<'one_two_three_four_five')" ; done

/bigstore/binfo/zebra_finch/RegionXExpression/Fastp/Blue623X_S6_L001_R1_001
/bigstore/binfo/zebra_finch/RegionXExpression/Fastp/Blue623X_S6_L001_R2_001
/bigstore/binfo/zebra_finch/RegionXExpression/Fastp/Blue623X_S6_L002_R1_001
/bigstore/binfo/zebra_finch/RegionXExpression/Fastp/Blue623X_S6_L002_R2_001
/bigstore/binfo/zebra_finch/RegionXExpression/Fastp/Brown443X_S79_L007_R1_001
/bigstore/binfo/zebra_finch/RegionXExpression/Fastp/Brown443X_S79_L007_R2_001
/bigstore/binfo/zebra_finch/RegionXExpression/Fastp/Brown443X_S79_L008_R1_001
/bigstore/binfo/zebra_finch/RegionXExpression/Fastp/Brown443X_S79_L008_R2_001
/bigstore/binfo/zebra_finch/RegionXExpression/Fastp/LtPink78X_S40_L003_R1_001
/bigstore/binfo/zebra_finch/RegionXExpression/Fastp/LtPink78X_S40_L003_R2_001
/bigstore/binfo/zebra_finch/RegionXExpression/Fastp/LtPink78X_S40_L004_R1_001
/bigstore/binfo/zebra_finch/RegionXExpression/Fastp/LtPink78X_S40_L004_R2_001
/bigstore/binfo/zebra_finch/RegionXExpression/Fastp/LtPink82X_S67_L007_R

# Cufflinks
    https://github.com/cole-trapnell-lab/cufflinks
    conda install -c bioconda cufflinks